In [97]:
#Import all dependencies
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [98]:
# Import scraped csv and create dataframes

players_file_path = Path('../scrapping/player_list.csv')
stats_file_path = Path('../scrapping/player_stats.csv')

players_list_df = pd.read_csv(players_file_path, low_memory=False)
players_stats_df = pd.read_csv(stats_file_path, low_memory=False)

# Transform players_stats_df

In [99]:
# Make a copy
player_stats_ready_df = players_stats_df.copy()

#Do simple transformations
player_stats_ready_df["player_id"] = player_stats_ready_df["player_id"].replace({'player_':''}, regex=True)
player_stats_ready_df["calendar"] = player_stats_ready_df["calendar"].replace({'_3_calendar':''}, regex=True)
player_stats_ready_df["calendar"] = player_stats_ready_df["calendar"].replace({'_2_calendar':''}, regex=True)
player_stats_ready_df["calendar"] = player_stats_ready_df["calendar"].replace({'_1_calendar':''}, regex=True)
player_stats_ready_df["calendar"] = player_stats_ready_df["calendar"].replace({'championship_':''}, regex=True)
player_stats_ready_df["match_id"] = player_stats_ready_df["match_id"].replace({'match_':''}, regex=True)
player_stats_ready_df["match_id"] = player_stats_ready_df["match_id"].replace({'_stats':''}, regex=True)
player_stats_ready_df["match_rate"] = player_stats_ready_df["match_rate"].replace({'-':'0'}, regex=True)
player_stats_ready_df["percentage_pass"] = player_stats_ready_df["percentage_pass"].replace({'-':'0'}, regex=True)

#Change types when needed
player_stats_ready_df["player_id"] = player_stats_ready_df["player_id"].astype(str).astype(int)
player_stats_ready_df["match_id"] = player_stats_ready_df["match_id"].astype(str).astype(int)
player_stats_ready_df["match_rate"] = player_stats_ready_df["match_rate"].astype(str).astype(float)
player_stats_ready_df["calendar"] = player_stats_ready_df["calendar"].astype(str).astype(int)
player_stats_ready_df["percentage_pass"] = player_stats_ready_df["percentage_pass"].astype(str).astype(float)

#Drop redundant columns
player_stats_ready_df = player_stats_ready_df.drop(["last_name"], axis=1)

# Replace NaN with 0
player_stats_ready_df = player_stats_ready_df.fillna(0)

# Tranform match_date
player_stats_ready_df["match_date"] = pd.to_datetime(player_stats_ready_df["match_date"])
player_stats_ready_df["match_date"] = player_stats_ready_df["match_date"].dt.date
player_stats_ready_df["match_date"] = pd.to_datetime(player_stats_ready_df["match_date"])

player_stats_ready_df

,player_id,calendar,number_of_matches,total_appearances,starter,sub_in,sub_out,match_date,season_day,match_id,goals,minutes_played,match_rate,sub,accurate_back_zone_pass,accurate_cross_nocorner,accurate_fwd_zone_pass,accurate_long_balls,accurate_pass,att_pen_goal,att_pen_miss,att_pen_post,att_pen_target,big_chance_created,big_chance_missed,clean_sheet,duel_lost,duel_won,error_lead_to_goal,failed_cross,fouls,goal_assist,goals_conceded,interception_won,lost_contest,own_goals,percentage_pass,poss_lost_ctrl,red_card,shot_off_target,total_back_zone_pass,total_contest,total_cross_nocorner,total_fwd_zone_pass,total_long_balls,total_pass,total_scoring_att,total_tackle,touches,was_fouled,won_contest,yellow_card,penalty_save,penalty_faced,saves,dive_save,stand_save
0,101148,2017,38,34,32,1,1,2018-05-13,38,919268,0.0,90,6.0,0,15.0,0,8.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.9,2.0,0,0.0,15.0,0.0,0.0,10.0,2.0,25.0,0.0,1.0,34.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,101148,2017,38,34,32,1,1,2018-05-05,37,919261,0.0,90,5.0,0,47.0,0,19.0,9.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,10.0,0,0.0,1.0,0.0,2,0.0,0.0,0.0,0.8,13.0,0,0.0,51.0,0.0,0.0,28.0,15.0,79.0,0.0,1.0,92.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,101148,2017,38,34,32,1,1,2018-04-28,36,919248,0.0,90,5.0,0,37.0,0,25.0,4.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.0,5.0,0,0.0,2.0,0.0,1,1.0,0.0,0.0,0.8,12.0,0,1.0,41.0,0.0,0.0,32.0,10.0,73.0,1.0,0.0,85.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,101148,2017,38,34,32,1,1,2018-04-23,35,919237,0.0,90,5.0,0,9.0,0,6.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,8.0,0,0.0,1.0,0.0,1,1.0,0.0,0.0,0.6,12.0,0,0.0,15.0,0.0,0.0,12.0,6.0,27.0,0.0,1.0,39.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,101148,2017,38,34,32,1,1,2018-04-15,34,919228,0.0,90,5.0,0,7.0,0,3.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,4.0,0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.5,11.0,0,0.0,11.0,0.0,0.0,10.0,6.0,21.0,0.0,0.0,30.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24545,98747,2019,29,29,29,0,0,2019-08-17,2,1059712,0.0,90,7.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,7.0,1.0,0.0
24546,98747,2019,29,29,29,0,0,2019-08-10,1,1059704,0.0,90,6.5,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3.0,1.0,1.0
24547,98770,2019,29,3,1,2,0,2020-01-12,22,1059913,0.0,90,3.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,5.0,0.0,0.0
24548,98770,2019,29,3,1,2,0,2020-01-01,21,1059904,0.0,15,5.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


# Merge players_stats_df and players_list_df

In [100]:
# Merge DataFrames
all_epl_players_ready_df = player_stats_ready_df.merge(players_list_df, how='left', on='player_id')
all_epl_players_ready_df

,player_id,calendar,number_of_matches,total_appearances,starter,sub_in,sub_out,match_date,season_day,match_id,goals,minutes_played,match_rate,sub,accurate_back_zone_pass,accurate_cross_nocorner,accurate_fwd_zone_pass,accurate_long_balls,accurate_pass,att_pen_goal,att_pen_miss,att_pen_post,att_pen_target,big_chance_created,big_chance_missed,clean_sheet,duel_lost,duel_won,error_lead_to_goal,failed_cross,fouls,goal_assist,goals_conceded,interception_won,lost_contest,own_goals,percentage_pass,poss_lost_ctrl,red_card,shot_off_target,total_back_zone_pass,total_contest,total_cross_nocorner,total_fwd_zone_pass,total_long_balls,total_pass,total_scoring_att,total_tackle,touches,was_fouled,won_contest,yellow_card,penalty_save,penalty_faced,saves,dive_save,stand_save,last_name,first_name,position,club,team_id
0,101148,2017,38,34,32,1,1,2018-05-13,38,919268,0.0,90,6.0,0,15.0,0,8.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.9,2.0,0,0.0,15.0,0.0,0.0,10.0,2.0,25.0,0.0,1.0,34.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,Lascelles,Jamaal,2,Newcastle,4
1,101148,2017,38,34,32,1,1,2018-05-05,37,919261,0.0,90,5.0,0,47.0,0,19.0,9.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,10.0,0,0.0,1.0,0.0,2,0.0,0.0,0.0,0.8,13.0,0,0.0,51.0,0.0,0.0,28.0,15.0,79.0,0.0,1.0,92.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,Lascelles,Jamaal,2,Newcastle,4
2,101148,2017,38,34,32,1,1,2018-04-28,36,919248,0.0,90,5.0,0,37.0,0,25.0,4.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.0,5.0,0,0.0,2.0,0.0,1,1.0,0.0,0.0,0.8,12.0,0,1.0,41.0,0.0,0.0,32.0,10.0,73.0,1.0,0.0,85.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,Lascelles,Jamaal,2,Newcastle,4
3,101148,2017,38,34,32,1,1,2018-04-23,35,919237,0.0,90,5.0,0,9.0,0,6.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,8.0,0,0.0,1.0,0.0,1,1.0,0.0,0.0,0.6,12.0,0,0.0,15.0,0.0,0.0,12.0,6.0,27.0,0.0,1.0,39.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,Lascelles,Jamaal,2,Newcastle,4
4,101148,2017,38,34,32,1,1,2018-04-15,34,919228,0.0,90,5.0,0,7.0,0,3.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,4.0,0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.5,11.0,0,0.0,11.0,0.0,0.0,10.0,6.0,21.0,0.0,0.0,30.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,Lascelles,Jamaal,2,Newcastle,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24545,98747,2019,29,29,29,0,0,2019-08-17,2,1059712,0.0,90,7.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,7.0,1.0,0.0,Pope,Nick,1,Burnley,90
24546,98747,2019,29,29,29,0,0,2019-08-10,1,1059704,0.0,90,6.5,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3.0,1.0,1.0,Pope,Nick,1,Burnley,90
24547,98770,2019,29,3,1,2,0,2020-01-12,22,1059913,0.0,90,3.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,5.0,0.0,0.0,Nyland,Ørjan,1,Aston Villa,7
24548,98770,2019,29,3,1,2,0,2020-01-01,21,1059904,0.0,15,5.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,Nyland,Ørjan,1,Aston Villa,7


In [101]:
# Reorganize columns
cols = ['player_id', 'first_name', 'last_name', 'position', 'team_id', 'club', 'calendar', 'number_of_matches', 'total_appearances', 'starter', 'sub_in', 'sub_out', 'match_id', 'season_day', 'match_date', 'minutes_played', 'match_rate', 'goals', 'sub', 'accurate_back_zone_pass', 'accurate_cross_nocorner', 'accurate_fwd_zone_pass', 'accurate_long_balls', 'accurate_pass', 'att_pen_goal', 'att_pen_miss', 'att_pen_post', 'att_pen_target', 'big_chance_created', 'big_chance_missed', 'clean_sheet', 'duel_lost', 'duel_won', 'error_lead_to_goal', 'failed_cross', 'fouls', 'goal_assist', 'goals_conceded', 'interception_won', 'lost_contest', 'own_goals', 'percentage_pass', 'poss_lost_ctrl', 'red_card', 'shot_off_target', 'total_back_zone_pass', 'total_contest', 'total_cross_nocorner', 'total_fwd_zone_pass', 'total_long_balls', 'total_pass', 'total_scoring_att', 'total_tackle', 'touches', 'was_fouled', 'won_contest', 'yellow_card', 'penalty_save', 'penalty_faced', 'saves', 'dive_save', 'stand_save']
all_epl_players_ready_df = all_epl_players_ready_df[cols]

# Reorder dataframe
all_epl_players_ready_df = all_epl_players_ready_df.sort_values(['calendar', 'club', 'position', 'last_name', 'season_day'], ascending=[1, 1, 1, 1, 1])
all_epl_players_ready_df = all_epl_players_ready_df.reset_index(drop=True)

# Save as csv
all_epl_players_ready_df.to_csv('all_epl_players_ready_df.csv', index=True)

all_epl_players_ready_df

,player_id,first_name,last_name,position,team_id,club,calendar,number_of_matches,total_appearances,starter,sub_in,sub_out,match_id,season_day,match_date,minutes_played,match_rate,goals,sub,accurate_back_zone_pass,accurate_cross_nocorner,accurate_fwd_zone_pass,accurate_long_balls,accurate_pass,att_pen_goal,att_pen_miss,att_pen_post,att_pen_target,big_chance_created,big_chance_missed,clean_sheet,duel_lost,duel_won,error_lead_to_goal,failed_cross,fouls,goal_assist,goals_conceded,interception_won,lost_contest,own_goals,percentage_pass,poss_lost_ctrl,red_card,shot_off_target,total_back_zone_pass,total_contest,total_cross_nocorner,total_fwd_zone_pass,total_long_balls,total_pass,total_scoring_att,total_tackle,touches,was_fouled,won_contest,yellow_card,penalty_save,penalty_faced,saves,dive_save,stand_save
0,98980,Emiliano,Martínez,1,3,Arsenal,2017,38,36,5,0,31,942801,1,2017-08-20,0,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,98980,Emiliano,Martínez,1,3,Arsenal,2017,38,36,5,0,31,942816,2,2017-08-27,0,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,98980,Emiliano,Martínez,1,3,Arsenal,2017,38,36,5,0,31,942823,3,2017-09-08,0,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,98980,Emiliano,Martínez,1,3,Arsenal,2017,38,36,5,0,31,942831,4,2017-09-16,0,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,98980,Emiliano,Martínez,1,3,Arsenal,2017,38,36,5,0,31,942839,5,2017-09-21,0,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24545,102057,Raúl,Jiménez,4,39,Wolverhampton,2019,29,29,28,1,0,1059947,25,2020-02-01,91,5.0,0.0,0,5.0,0,13.0,2.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7.0,7.0,0,0.0,2.0,0.0,0,1.0,1.0,0.0,0.8,10.0,0,0.0,5.0,5.0,0.0,19.0,5.0,24.0,1.0,1.0,41.0,2.0,4.0,0,0.0,0.0,0.0,0.0,0.0
24546,102057,Raúl,Jiménez,4,39,Wolverhampton,2019,29,29,28,1,0,1059961,26,2020-02-14,90,5.0,0.0,0,7.0,0,15.0,1.0,22.0,0.0,0.0,0.0,0.0,0.0,2.0,1,3.0,10.0,0,1.0,2.0,0.0,0,0.0,1.0,0.0,0.7,15.0,0,4.0,10.0,5.0,1.0,22.0,1.0,31.0,5.0,1.0,50.0,2.0,4.0,0,0.0,0.0,0.0,0.0,0.0
24547,102057,Raúl,Jiménez,4,39,Wolverhampton,2019,29,29,28,1,0,1059971,27,2020-02-23,72,6.0,1.0,0,5.0,0,12.0,1.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0,10.0,2.0,0,1.0,3.0,0.0,0,1.0,1.0,0.0,0.7,16.0,0,3.0,6.0,2.0,1.0,18.0,3.0,23.0,4.0,0.0,42.0,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0
24548,102057,Raúl,Jiménez,4,39,Wolverhampton,2019,29,29,28,1,0,1059979,28,2020-03-01,92,7.0,1.0,0,6.0,0,13.0,1.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0,9.0,8.0,0,1.0,1.0,0.0,2,1.0,1.0,0.0,0.7,15.0,0,2.0,7.0,4.0,1.0,22.0,1.0,28.0,5.0,1.0,48.0,0.0,3.0,0,0.0,0.0,0.0,0.0,0.0


# Extracting forward matches with >45 mns played

In [102]:
# Filter DataFrame
forward_epl_players_ready_df = all_epl_players_ready_df[(all_epl_players_ready_df["position"]==4) & (all_epl_players_ready_df["minutes_played"]>45)]
forward_epl_players_ready_df = forward_epl_players_ready_df.reset_index(drop=True)

# Save as csv
forward_epl_players_ready_df.to_csv('forward_epl_players_ready_df.csv', index=True)

forward_epl_players_ready_df


,player_id,first_name,last_name,position,team_id,club,calendar,number_of_matches,total_appearances,starter,sub_in,sub_out,match_id,season_day,match_date,minutes_played,match_rate,goals,sub,accurate_back_zone_pass,accurate_cross_nocorner,accurate_fwd_zone_pass,accurate_long_balls,accurate_pass,att_pen_goal,att_pen_miss,att_pen_post,att_pen_target,big_chance_created,big_chance_missed,clean_sheet,duel_lost,duel_won,error_lead_to_goal,failed_cross,fouls,goal_assist,goals_conceded,interception_won,lost_contest,own_goals,percentage_pass,poss_lost_ctrl,red_card,shot_off_target,total_back_zone_pass,total_contest,total_cross_nocorner,total_fwd_zone_pass,total_long_balls,total_pass,total_scoring_att,total_tackle,touches,was_fouled,won_contest,yellow_card,penalty_save,penalty_faced,saves,dive_save,stand_save
0,54694,Pierre-Emerick,Aubameyang,4,3,Arsenal,2017,38,13,12,1,0,919144,26,2018-02-03,90,7.0,1.0,0,10.0,0,5.0,1.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,3.0,0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.9,4.0,0,0.0,10.0,0.0,0.0,7.0,1.0,17.0,3.0,1.0,25.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,54694,Pierre-Emerick,Aubameyang,4,3,Arsenal,2017,38,13,12,1,0,919161,27,2018-02-10,90,4.0,0.0,0,6.0,0,4.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0,10.0,1.0,0,2.0,1.0,0.0,1,1.0,0.0,0.0,0.6,10.0,0,0.0,8.0,0.0,2.0,12.0,0.0,18.0,0.0,0.0,23.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,54694,Pierre-Emerick,Aubameyang,4,3,Arsenal,2017,38,13,12,1,0,919164,28,2018-03-01,90,2.0,0.0,0,6.0,0,5.0,0.0,11.0,0.0,0.0,0.0,1.0,0.0,1.0,0,4.0,2.0,0,2.0,0.0,0.0,3,0.0,0.0,0.0,0.6,12.0,0,1.0,9.0,2.0,2.0,13.0,1.0,20.0,2.0,0.0,27.0,0.0,2.0,0,0.0,0.0,0.0,0.0,0.0
3,54694,Pierre-Emerick,Aubameyang,4,3,Arsenal,2017,38,13,12,1,0,919173,29,2018-03-04,90,6.0,1.0,0,8.0,0,7.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7.0,4.0,0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.7,8.0,0,0.0,9.0,1.0,0.0,12.0,0.0,21.0,2.0,1.0,29.0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0
4,54694,Pierre-Emerick,Aubameyang,4,3,Arsenal,2017,38,13,12,1,0,919184,30,2018-03-11,90,7.0,1.0,0,3.0,0,5.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,2.0,1,7.0,2.0,0,0.0,0.0,1.0,0,0.0,1.0,0.0,0.8,6.0,0,2.0,4.0,2.0,0.0,6.0,0.0,10.0,4.0,0.0,24.0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,102057,Raúl,Jiménez,4,39,Wolverhampton,2019,29,29,28,1,0,1059947,25,2020-02-01,91,5.0,0.0,0,5.0,0,13.0,2.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7.0,7.0,0,0.0,2.0,0.0,0,1.0,1.0,0.0,0.8,10.0,0,0.0,5.0,5.0,0.0,19.0,5.0,24.0,1.0,1.0,41.0,2.0,4.0,0,0.0,0.0,0.0,0.0,0.0
3115,102057,Raúl,Jiménez,4,39,Wolverhampton,2019,29,29,28,1,0,1059961,26,2020-02-14,90,5.0,0.0,0,7.0,0,15.0,1.0,22.0,0.0,0.0,0.0,0.0,0.0,2.0,1,3.0,10.0,0,1.0,2.0,0.0,0,0.0,1.0,0.0,0.7,15.0,0,4.0,10.0,5.0,1.0,22.0,1.0,31.0,5.0,1.0,50.0,2.0,4.0,0,0.0,0.0,0.0,0.0,0.0
3116,102057,Raúl,Jiménez,4,39,Wolverhampton,2019,29,29,28,1,0,1059971,27,2020-02-23,72,6.0,1.0,0,5.0,0,12.0,1.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0,10.0,2.0,0,1.0,3.0,0.0,0,1.0,1.0,0.0,0.7,16.0,0,3.0,6.0,2.0,1.0,18.0,3.0,23.0,4.0,0.0,42.0,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0
3117,102057,Raúl,Jiménez,4,39,Wolverhampton,2019,29,29,28,1,0,1059979,28,2020-03-01,92,7.0,1.0,0,6.0,0,13.0,1.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0,9.0,8.0,0,1.0,1.0,0.0,2,1.0,1.0,0.0,0.7,15.0,0,2.0,7.0,4.0,1.0,22.0,1.0,28.0,5.0,1.0,48.0,0.0,3.0,0,0.0,0.0,0.0,0.0,0.0
